In [0]:
ad_feature = spark.read.format("csv").option("inferSchema", True).option("header", True).option("sep", ',').option("path", "dbfs:/tmp/ad_feature.csv").load()
raw_sample = spark.read.format("csv").option("inferSchema", True).option("header", True).option("sep", ',').option("path", "dbfs:/tmp/raw_sample.csv").load()
user_profile = spark.read.format("csv").option("inferSchema", True).option("header", True).option("sep", ',').option("path", "dbfs:/tmp/user_profile.csv").load()
behavior_log = spark.read.format("csv").option("inferSchema", True).option("header", True).option("sep", ',').option("path", "dbfs:/tmp/behavior_log.csv").load()

In [0]:
behavior_log.show(5)

+------+----------+----+-----+------+
 user|time_stamp|btag| cate| brand|
+------+----------+----+-----+------+
558157|1493741625| pv| 6250| 91286|
558157|1493741626| pv| 6250| 91286|
558157|1493741627| pv| 6250| 91286|
728690|1493776998| pv|11800| 62353|
332634|1493809895| pv| 1101|365477|
+------+----------+----+-----+------+
only showing top 5 rows

In [0]:
from datetime import datetime
from pyspark.sql.functions import *
behavior_log = behavior_log.withColumn('bh_hour', UserDefinedFunction(lambda obj: datetime.fromtimestamp(obj).hour)(behavior_log.time_stamp))
behavior_log = behavior_log.drop("hour")
behavior_log.show()

+------+----------+----+-----+------+-------+
 user|time_stamp|btag| cate| brand|bh_hour|
+------+----------+----+-----+------+-------+
558157|1493741625| pv| 6250| 91286| 16|
558157|1493741626| pv| 6250| 91286| 16|
558157|1493741627| pv| 6250| 91286| 16|
728690|1493776998| pv|11800| 62353| 2|
332634|1493809895| pv| 1101|365477| 11|
857237|1493816945| pv| 1043|110616| 13|
619381|1493774638| pv| 385|428950| 1|
467042|1493772641| pv| 8237|301299| 0|
467042|1493772644| pv| 8237|301299| 0|
991528|1493780710| pv| 7270|274795| 3|
991528|1493780712| pv| 7270|274795| 3|
991528|1493780712| pv| 7270|274795| 3|
991528|1493780712| pv| 7270|274795| 3|
991528|1493780714| pv| 7270|274795| 3|
991528|1493780765| pv| 7270|274795| 3|
991528|1493780714| pv| 7270|274795| 3|
991528|1493780765| pv| 7270|274795| 3|
991528|1493780764| pv| 7270|274795| 3|
991528|1493780633| pv| 7270|274795| 3|
991528|1493780764| pv| 7270|274795| 3|
+------+----------+----+-----+------+-------+
only showing top 20 rows

In [0]:
spark.sql("select user, row_number() over(partition by user order by bh_hour desc ) as rankid from behavior_log").show()

In [0]:
# find the mode of hour for each user
behavior_log.createOrReplaceTempView("behavior_log")
bh_hour = spark.sql("select * from (select *, ROW_NUMBER() over(PARTITION BY user ORDER BY cnt DESC) as rank_hour from (SELECT user, bh_hour, count(*) as cnt from behavior_log group by user, bh_hour) as kk) as gg where rank_hour=1 ")
bh_hour = bh_hour.drop("cnt","rank_hour")
bh_hour.show()

+----+-------+
user|bh_hour|
+----+-------+
 31| 13|
 53| 3|
 65| 12|
 85| 12|
 137| 13|
 148| 16|
 243| 4|
 251| 2|
 255| 23|
 392| 6|
 451| 14|
 458| 12|
 463| 18|
 471| 15|
 481| 11|
 496| 14|
 516| 6|
 540| 22|
 580| 9|
 588| 2|
+----+-------+
only showing top 20 rows

In [0]:
user_profile.createOrReplaceTempView("user_profile")
bh_hour.createOrReplaceTempView("bh_hour")
user_profile = spark.sql("select * from user_profile as a join bh_hour as b on a.userid = b.user")
user_profile.show()

+------+---------+------------+-----------------+---------+------------+--------------+----------+---------------------+----+-------+
userid|cms_segid|cms_group_id|final_gender_code|age_level|pvalue_level|shopping_level|occupation|new_user_class_level |user|bh_hour|
+------+---------+------------+-----------------+---------+------------+--------------+----------+---------------------+----+-------+
 31| 6| 2| 2| 2| 1| 1| 0| null| 31| 13|
 148| 0| 11| 1| 5| null| 3| 0| 2| 148| 16|
 243| 0| 11| 1| 5| null| 3| 0| null| 243| 4|
 392| 0| 9| 1| 3| null| 3| 0| 2| 392| 6|
 463| 19| 3| 2| 3| 2| 3| 0| 2| 463| 18|
 471| 18| 3| 2| 3| 2| 3| 0| 1| 471| 15|
 496| 0| 4| 2| 4| null| 3| 0| 3| 496| 14|
 516| 0| 3| 2| 3| null| 3| 0| 2| 516| 6|
 540| 0| 4| 2| 4| null| 3| 0| 2| 540| 22|
 623| 0| 10| 1| 4| null| 3| 0| 4| 623| 13|
 737| 0| 5| 2| 5| null| 3| 0| null| 737| 13|
 833| 0| 1| 2| 1| null| 2| 0| null| 833| 14|
 858| 0| 5| 2| 5| null| 3| 0| null| 858| 5|
 897| 0| 3| 2| 3| null| 3| 0| 2| 897| 7|
 1025| 0| 3| 2| 3| null| 3| 0| 4|1025| 14|
 1084| 67| 9| 1| 3| 2| 3| 0| 3|1084| 8|
 1088| 0| 4| 2| 4| 2| 3| 0| null|1088| 13|
 1127| 0| 2| 2| 2| null| 2| 0| null|1127| 16|
 1139| 0| 3| 2| 3| 2| 3| 1| null|1139| 7|
 1143| 0| 10| 1| 4| null| 3| 0| null|1143| 16|
+------+---------+------------+-----------------+---------+------------+--------------+----------+---------------------+----+-------+
only showing top 20 rows

In [0]:
last_three_days = spark.sql("select * from behavior_log where time_stamp > 1494302400")
last_three_days.count() / behavior_log.count()

Out[72]: 0.23315548559754465

In [0]:
last_three_days.show()

+------+----------+----+-----+------+-------+
 user|time_stamp|btag| cate| brand|bh_hour|
+------+----------+----+-----+------+-------+
537182|1725192889| pv|10579|355626| 12|
 92282|1662052030| pv| 4278| 83456| 17|
179060|1985024574| pv| 5945|189678| 19|
179060|1985024250| pv| 5945|189678| 19|
638786|1630625225| pv| 9447|374881| 23|
638786|1630625506| pv| 9447|374881| 23|
638786|1630625433| pv| 9447|374881| 23|
638786|1630625419| pv| 9447|374881| 23|
638786|1630625415| pv| 9447|374881| 23|
638786|1630625413| pv| 9447|374881| 23|
638786|1630625221| pv| 9447|374881| 23|
179060|1985027850| pv| 5945|184921| 20|
424111|1662050000| pv| 6412| 38728| 16|
638786|1630627882| pv| 6426|310886| 0|
638786|1630627868| pv| 6426|310886| 0|
504560|1725335479| pv| 6064|340277| 3|
179060|1985031826| pv| 5945|152985| 21|
179060|1985031896| pv| 5945|291982| 21|
638786|1630620665| pv| 6261|318890| 22|
179060|1985029269| pv| 5945| 76603| 21|
+------+----------+----+-----+------+-------+
only showing top 20 rows

In [0]:
user_profile.createOrReplaceTempView("user_profile")
last_three_days.createOrReplaceTempView("last_three_days")
personal_cate = spark.sql("select user, cate as personal_cate from (select *, ROW_NUMBER() over(PARTITION BY user ORDER BY cnt DESC) as rank_cate from (SELECT user, cate, count(*) as cnt from last_three_days group by user, cate) as kk) as gg where rank_cate=1")
personal_brand = spark.sql("select user, brand as personal_brand from (select *, ROW_NUMBER() over(PARTITION BY user ORDER BY cnt DESC) as rank_brand from (SELECT user, brand, count(*) as cnt from last_three_days group by user, brand) as kk) as gg where rank_brand=1")

In [0]:
personal_cate.show()

+----+-------------+
user|personal_cate|
+----+-------------+
 31| 6261|
 148| 9899|
 243| 4595|
 392| 4826|
 463| 6261|
 471| 6261|
 496| 6261|
 516| 6426|
 540| 4846|
 623| 5474|
 737| 7268|
 833| 6261|
 858| 5343|
 897| 1456|
1025| 6261|
1084| 9443|
1088| 4526|
1139| 6261|
1143| 4283|
1238| 6407|
+----+-------------+
only showing top 20 rows

In [0]:
personal_cate.createOrReplaceTempView("personal_cate")
personal_brand.createOrReplaceTempView("personal_brand")
user_profile = spark.sql("select * from user_profile as a join personal_cate as b on a.userid = b.user")
user_profile.createOrReplaceTempView("user_profile")
user_profile = spark.sql("select * from user_profile as a join personal_brand as b on a.userid = b.user")
user_profile.show()

+------+---------+------------+-----------------+---------+------------+--------------+----------+---------------------+----+-------+----+-------------+----+--------------+
userid|cms_segid|cms_group_id|final_gender_code|age_level|pvalue_level|shopping_level|occupation|new_user_class_level |user|bh_hour|user|personal_cate|user|personal_brand|
+------+---------+------------+-----------------+---------+------------+--------------+----------+---------------------+----+-------+----+-------------+----+--------------+
 31| 6| 2| 2| 2| 1| 1| 0| null| 31| 13| 31| 6261| 31| 334705|
 148| 0| 11| 1| 5| null| 3| 0| 2| 148| 16| 148| 9899| 148| 356337|
 243| 0| 11| 1| 5| null| 3| 0| null| 243| 4| 243| 4595| 243| 272582|
 392| 0| 9| 1| 3| null| 3| 0| 2| 392| 6| 392| 4826| 392| 388021|
 463| 19| 3| 2| 3| 2| 3| 0| 2| 463| 18| 463| 6261| 463| 115961|
 471| 18| 3| 2| 3| 2| 3| 0| 1| 471| 15| 471| 6261| 471| 124895|
 496| 0| 4| 2| 4| null| 3| 0| 3| 496| 14| 496| 6261| 496| 372358|
 516| 0| 3| 2| 3| null| 3| 0| 2| 516| 6| 516| 6426| 516| 64569|
 540| 0| 4| 2| 4| null| 3| 0| 2| 540| 22| 540| 4846| 540| 446182|
 623| 0| 10| 1| 4| null| 3| 0| 4| 623| 13| 623| 5474| 623| 98931|
 737| 0| 5| 2| 5| null| 3| 0| null| 737| 13| 737| 7268| 737| 349143|
 833| 0| 1| 2| 1| null| 2| 0| null| 833| 14| 833| 6261| 833| 18595|
 858| 0| 5| 2| 5| null| 3| 0| null| 858| 5| 858| 5343| 858| 461203|
 897| 0| 3| 2| 3| null| 3| 0| 2| 897| 7| 897| 1456| 897| 255149|
 1025| 0| 3| 2| 3| null| 3| 0| 4|1025| 14|1025| 6261|1025| 313882|
 1084| 67| 9| 1| 3| 2| 3| 0| 3|1084| 8|1084| 9443|1084| 95036|
 1088| 0| 4| 2| 4| 2| 3| 0| null|1088| 13|1088| 4526|1088| 198567|
 1139| 0| 3| 2| 3| 2| 3| 1| null|1139| 7|1139| 6261|1139| 185298|
 1143| 0| 10| 1| 4| null| 3| 0| null|1143| 16|1143| 4283|1143| 419236|
 1238| 0| 5| 2| 5| null| 3| 0| 3|1238| 23|1238| 6407|1238| 315834|
+------+---------+------------+-----------------+---------+------------+--------------+----------+---------------------+----+-------+----+-------------+----+--------------+
only showing top 20 rows

In [0]:
behavior_log_without_pv = spark.sql("select * from behavior_log where btag != 'pv'")
behavior_log_without_pv.show()

+-------+----------+----+-----+------+-------+
 user|time_stamp|btag| cate| brand|bh_hour|
+-------+----------+----+-----+------+-------+
 840457|1493741837|cart| 2155|355993| 16|
 840457|1493741844|cart| 2155|355993| 16|
 832679|1493776906| fav| 5515|245576| 2|
 916958|1493811673| buy| 123| 97176| 11|
 916958|1493811809| buy| 123| 97176| 11|
 95085|1493822768|cart|10076| 78067| 14|
 436581|1493803451|cart| 4587|158673| 9|
 511672|1493786222| fav| 6982|426734| 4|
 619453|1493814145|cart| 4778| 12908| 12|
 127524|1493826608| fav| 7205|167727| 15|
 848967|1493768758|cart| 9460|316040| 23|
 628854|1493781424| buy| 4262|370203| 3|
 623496|1493810730| buy| 4262|370203| 11|
 44083|1493790199| fav| 7208|289629| 5|
 797847|1493788224|cart| 9390|347564| 5|
 476887|1493801973|cart| 9390|347564| 8|
 355991|1493781379|cart| 9390|347564| 3|
1099015|1493774927| buy| 9390|347564| 1|
 752294|1493783841|cart| 6606| 33021| 3|
 543872|1493795299| buy| 6606| 33021| 7|
+-------+----------+----+-----+------+-------+
only showing top 20 rows

In [0]:
behavior_log_without_pv.createOrReplaceTempView("behavior_log_without_pv")
brand_influence = spark.sql("select brand, count(*) as brand_influence from behavior_log_without_pv group by brand")
brand_influence.show()

+------+---------------+
 brand|brand_influence|
+------+---------------+
243119| 12778|
120899| 2451|
457587| 704|
135000| 254|
 73091| 39|
 87120| 12895|
 53565| 77|
274848| 60|
447071| 39|
195004| 98|
249376| 133|
180753| 565|
413891| 480|
158339| 1252|
311544| 15|
385313| 566|
344070| 154|
386599| 3|
152600| 3229|
265363| 119|
+------+---------------+
only showing top 20 rows

In [0]:
bc_influence = spark.sql("select brand, cate, count(*) as bc_influence from behavior_log_without_pv group by brand, cate")
bc_influence.show()

+------+-----+------------+
 brand| cate|bc_influence|
+------+-----+------------+
181024| 4284| 230|
 31630| 6160| 11|
 57342| 4385| 956|
 88981| 6441| 2|
198895|10733| 33|
185837| 6468| 82|
176515| 4282| 858|
460873| 5497| 21|
253021| 6423| 214|
 7918| 1101| 26|
229351| 2181| 122|
245188| 6794| 20|
258254| 4290| 3078|
438964| 6845| 16|
176864| 6261| 2496|
370203| 9841| 707|
293023| 5238| 330|
 60687| 8998| 194|
374220| 5950| 7|
357648| 6736| 31|
+------+-----+------------+
only showing top 20 rows

In [0]:
ad_feature.show(5)

In [0]:
ad_feature = spark.read.format("csv").option("inferSchema", True).option("header", True).option("sep", ',').option("path", "dbfs:/tmp/ad_feature.csv").load()
ad_feature.createOrReplaceTempView("ad_feature")
brand_influence.createOrReplaceTempView("brand_influence")
bc_influence.createOrReplaceTempView("bc_influence")
ad_feature = spark.sql("select adgroup_id, cate_id, campaign_id, customer, a.brand, price, brand_influence from ad_feature as a join brand_influence as b on a.brand = b.brand")
ad_feature.createOrReplaceTempView("ad_feature")
ad_feature = spark.sql("select adgroup_id, a.cate_id, campaign_id, customer, a.brand, price, brand_influence, b.cate, bc_influence from ad_feature as a join bc_influence as b on a.brand = b.brand and a.cate_id = b.cate")
ad_feature.show(5)

+----------+-------+-----------+--------+-----+------+---------------+----+------------+
adgroup_id|cate_id|campaign_id|customer|brand| price|brand_influence|cate|bc_influence|
+----------+-------+-----------+--------+-----+------+---------------+----+------------+
 67774| 9688| 158952| 40762| 3|2560.0| 432|9688| 12|
 18390| 9688| 3088| 40762| 3|2560.0| 432|9688| 12|
 582243| 9688| 364762| 29022| 3|2980.0| 432|9688| 12|
 115964| 9688| 397184| 51739| 3|4252.0| 432|9688| 12|
 3364| 3303| 415895| 30812| 43| 0.04| 1419|3303| 6|
+----------+-------+-----------+--------+-----+------+---------------+----+------------+
only showing top 5 rows

In [0]:
ad_feature.createOrReplaceTempView("ad_feature")
brand = spark.sql("select brand, btag, count(*) as cnt from behavior_log group by brand, btag order by brand, btag")
brand = brand.groupby("brand").pivot("btag").agg(sum("cnt"))
brand = brand.selectExpr("brand", "buy / pv as buy_pv_brand", "cart / pv as cart_pv_brand", "fav / pv as fav_pv_brand")
brand.show()

+------+--------------------+--------------------+--------------------+
 brand| buy_pv_brand| cart_pv_brand| fav_pv_brand|
+------+--------------------+--------------------+--------------------+
146224| null| null| 0.05263157894736842|
146411|0.007574163686092994| 0.02377445823690301|0.020197769829581318|
146464|0.002417794970986...| 0.01402321083172147|0.011121856866537718|
146581| null| null| 0.03508771929824561|
146988| 0.3333333333333333| null| null|
147280|0.015345268542199489| 0.04092071611253197|0.010230179028132993|
147524|0.004149377593360996|0.029045643153526972|0.014937759336099586|
147711|0.015384615384615385|0.015384615384615385| 0.03076923076923077|
147891| 0.4| null| null|
147958| null| null| 0.14285714285714285|
147961|0.003802281368821...|0.003802281368821...|0.015209125475285171|
148053|0.011320754716981131|0.022641509433962263| 0.03018867924528302|
148267|0.018329938900203666| 0.02240325865580448|0.014256619144602852|
148324| null| null| null|
149177|0.015209125475285171|0.015209125475285171|0.030418250950570342|
149465| 0.03233830845771144|0.017412935323383085|0.002487562189054...|
149688| null| null| null|
149761| null| null| null|
149764|0.023076923076923078|0.038461538461538464|0.015384615384615385|
149785|0.003211009174311927|0.011467889908256881|0.016972477064220184|
+------+--------------------+--------------------+--------------------+
only showing top 20 rows

In [0]:
brand.createOrReplaceTempView("brand")
ad_feature = spark.sql("select adgroup_id, a.brand, cate, campaign_id, customer, price, brand_influence, bc_influence, buy_pv_brand, cart_pv_brand, fav_pv_brand from ad_feature as a join brand as b on a.brand = b.brand")
ad_feature.show(5)

+----------+-----+----+-----------+--------+------+---------------+------------+--------------------+--------------------+--------------------+
adgroup_id|brand|cate|campaign_id|customer| price|brand_influence|bc_influence| buy_pv_brand| cart_pv_brand| fav_pv_brand|
+----------+-----+----+-----------+--------+------+---------------+------------+--------------------+--------------------+--------------------+
 624430|66166|9447| 94757| 15222| 798.0| 68| 6| null|0.013601741022850925|0.023394994559303592|
 114184|66166|9447| 87441| 15222| 376.0| 68| 6| null|0.013601741022850925|0.023394994559303592|
 118211|67089|4280| 218317| 30541| 359.0| 31| 12| null|0.020594965675057208|0.014874141876430207|
 24621|89537|5860| 9347| 87907|1300.0| 1075| 296|0.006786071865613527|0.008543775725887196|0.008588274557792859|
 343733|89537|5860| 325843| 87907|1300.0| 1075| 296|0.006786071865613527|0.008543775725887196|0.008588274557792859|
+----------+-----+----+-----------+--------+------+---------------+------------+--------------------+--------------------+--------------------+
only showing top 5 rows

In [0]:
bc = spark.sql("select brand, cate, btag, count(*) as cnt from behavior_log group by brand, cate, btag order by brand, cate, btag")
bc = bc.groupby("brand", "cate").pivot("btag").agg(sum("cnt"))
bc = bc.selectExpr("brand", "cate", "buy / pv as buy_pv_bc", "cart / pv as cart_pv_bc", "fav / pv as fav_pv_bc")
bc.show()

+-----+-----+-------------------+--------------------+--------------------+
brand| cate| buy_pv_bc| cart_pv_bc| fav_pv_bc|
+-----+-----+-------------------+--------------------+--------------------+
36349| 7177|0.07692307692307693| 0.07692307692307693| null|
36412| 6733| null| null| null|
36412|10324| null| null| null|
36453| 7261| null| null| null|
36463| 7970|0.04849884526558892| 0.03579676674364896|0.008083140877598153|
36515| 859| null| null| null|
36532| 6249| null| null| null|
36568| 6170| null| null| null|
36581| 4363| null| null| null|
36581| 8074| null| null|0.020618556701030927|
36653| 8169| null| null| null|
36658| 6142| null| null| null|
36684|11622| null| null| null|
36688| 5215| null| null| null|
36844|11590| null| null| null|
36862| 2159| null| null| null|
36897| 877| null| 0.01020408163265306|0.007653061224489796|
36964| 1762| null| null| null|
36964|11740| null|0.043478260869565216|0.021739130434782608|
37045| 9879| null| null| null|
+-----+-----+-------------------+--------------------+--------------------+
only showing top 20 rows

In [0]:
ad_feature.createOrReplaceTempView("ad_feature")
bc.createOrReplaceTempView("bc")
ad_feature = spark.sql("select adgroup_id, a.brand, a.cate, campaign_id, customer, price, brand_influence, bc_influence, buy_pv_brand, cart_pv_brand, fav_pv_brand, buy_pv_bc, cart_pv_bc, fav_pv_bc from ad_feature as a join bc as b on a.brand = b.brand and a.cate = b.cate")
ad_feature.show(5)

+----------+-----+----+-----------+--------+------+---------------+------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+
adgroup_id|brand|cate|campaign_id|customer| price|brand_influence|bc_influence| buy_pv_brand| cart_pv_brand| fav_pv_brand| buy_pv_bc| cart_pv_bc| fav_pv_bc|
+----------+-----+----+-----------+--------+------+---------------+------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+
 582243| 3|9688| 364762| 29022|2980.0| 432| 12|0.003726972633372378|0.02470450431263976| 0.01757001384304121| null| 0.0125|0.008928571428571428|
 67774| 3|9688| 158952| 40762|2560.0| 432| 12|0.003726972633372378|0.02470450431263976| 0.01757001384304121| null| 0.0125|0.008928571428571428|
 18390| 3|9688| 3088| 40762|2560.0| 432| 12|0.003726972633372378|0.02470450431263976| 0.01757001384304121| null| 0.0125|0.008928571428571428|
 115964| 3|9688| 397184| 51739|4252.0| 432| 12|0.003726972633372378|0.02470450431263976| 0.01757001384304121| null| 0.0125|0.008928571428571428|
 3364| 43|3303| 415895| 30812| 0.04| 1419| 6| 0.01410033347832391|0.02406843555168914|0.013266637668551544|0.041666666666666664|0.027777777777777776|0.013888888888888888|
+----------+-----+----+-----------+--------+------+---------------+------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 5 rows

In [0]:
ad_feature.cache()
user_profile.cache()

Out[122]: DataFrame[userid: int, cms_segid: int, cms_group_id: int, final_gender_code: int, age_level: int, pvalue_level: int, shopping_level: int, occupation: int, new_user_class_level : int, user: int, bh_hour: string, user: int, personal_cate: int, user: int, personal_brand: int]

In [0]:
ad_feature.coalesce(1).write.csv("/FileStore/ad_feature_zhu",header=True)

In [0]:
ad_feature.show()

+----------+-----+-----+-----------+--------+-----+---------------+------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
adgroup_id|brand| cate|campaign_id|customer|price|brand_influence|bc_influence| buy_pv_brand| cart_pv_brand| fav_pv_brand| buy_pv_bc| cart_pv_bc| fav_pv_bc|
+----------+-----+-----+-----------+--------+-----+---------------+------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
 3364| 43| 3303| 415895| 30812| 0.04| 1419| 6| 0.01410033347832391| 0.02406843555168914|0.013266637668551544|0.041666666666666664|0.027777777777777776|0.013888888888888888|
 6167| 43| 3303| 415895| 30812| 0.04| 1419| 6| 0.01410033347832391| 0.02406843555168914|0.013266637668551544|0.041666666666666664|0.027777777777777776|0.013888888888888888|
 226211| 43|10761| 166071| 49609| 1.5| 1419| 2| 0.01410033347832391| 0.02406843555168914|0.013266637668551544| 0.05555555555555555| 0.05555555555555555| null|
 6164| 701| 6427| 140825| 16429| 15.0| 672| 92|0.008360621472862817|0.021667943983836133|0.016790914791332823| 0.01729106628242075| 0.03025936599423631|0.018731988472622477|
 107231| 1754| 9713| 178674| 185529| 26.0| 49| 43|0.009933774834437087|0.015728476821192054|0.014900662251655629|0.009727626459143969|0.017509727626459144|0.014591439688715954|
 583229| 2226| 4301| 208221| 205755|198.0| 26| 5| 0.01607717041800643| 0.04180064308681672| 0.02572347266881029| null|0.047619047619047616|0.011904761904761904|
 774388| 3338| 6244| 368546| 247596| 39.8| 451| 56|0.012425575977219777| 0.02989904219518509| 0.01604970230390888|0.005529225908372828|0.022116903633491312|0.016587677725118485|
 133895| 3411| 910| 157638| 176914| 16.8| 13| 12|0.014534883720930232| 0.00872093023255814|0.014534883720930232|0.012944983818770227|0.009708737864077669|0.016181229773462782|
 66937| 3411| 910| 157638| 176914| 16.8| 13| 12|0.014534883720930232| 0.00872093023255814|0.014534883720930232|0.012944983818770227|0.009708737864077669|0.016181229773462782|
 345916| 3465| 6795| 16581| 93872| 8.8| 654| 58| 0.02229597481065441| 0.02135988426516892|0.011998978810314016| 0.03528114663726571|0.023153252480705624|0.005512679162072767|
 540859| 5172| 6735| 145850| 115076|113.8| 472| 433|0.016840277777777777| 0.04670138888888889| 0.01840277777777778| 0.01901294498381877| 0.05016181229773463|0.018406148867313916|
 613015| 6559| 6421| 357750| 192265|288.0| 2145| 56|0.014496955639315743|0.021700827441621873|0.011642170528804336|0.006344171292624901|0.017446471054718478|0.020618556701030927|
 561013| 6844| 6737| 301161| 237155|628.0| 546| 3|0.005240956154927777| 0.01476415697302825|0.014891985171928928| null|0.016666666666666666| 0.03333333333333333|
 513771| 6844| 6737| 301161| 237155|628.0| 546| 3|0.005240956154927777| 0.01476415697302825|0.014891985171928928| null|0.016666666666666666| 0.03333333333333333|
 76098| 7971|11571| 151061| 175696| 98.0| 68| 3|0.010430247718383311|0.014341590612777053|0.004780530204259...| null| 0.04838709677419355| null|
 176828| 7971|11571| 223578| 179999| 65.0| 68| 3|0.010430247718383311|0.014341590612777053|0.004780530204259...| null| 0.04838709677419355| null|
 470729| 8018| 5954| 139395| 426|178.0| 1310| 20|0.009059947756505638|0.018648943557186787|0.015606917303177596|0.002309468822170...|0.020785219399538105|0.023094688221709007|
 495487| 8266| 1552| 52018| 128993| 53.6| 7| 6|0.015873015873015872|0.015873015873015872|0.023809523809523808| 0.01834862385321101| 0.01834862385321101| 0.01834862385321101|
 640640| 9336| 6261| 226085| 214184|238.0| 655| 314|0.002740899357601713|0.010406852248394004|0.014903640256959314|0.001048034934497...| 0.01056768558951965|0.015807860262008734|
 407923| 9920| 5954| 202297| 8064|119.0| 624| 163|0.003676470588235294| 0.0198670814479638|0.020574095022624434| 0.00645819170632223|0.022433718558803536|0.026512576478585997|
+----------+-----+-----+------

In [0]:
# user_profile = user_profile.drop("user")
# user_profile.show(5)
user_profile.coalesce(1).write.csv("/FileStore/user_profile_zhu",header = 'true')